In [ ]:
import pandas as pd
import numpy as np

from pymaid_creds import url, name, password, token
import pymaid
rm = pymaid.CatmaidInstance(url, token, name, password)

# pull predicted NT data
NT_data = pd.read_csv('data/NT/all_brain_presynapses_preds_full.csv', index_col=0)

# pull all brain connectors
brain_skids = pymaid.get_skids_by_annotation('mw brain neurons') + pymaid.get_skids_by_annotation('mw brain accessory neurons')


In [20]:
# load connectors and determine fraction of each NT per neuron

connectors = pymaid.get_connector_details(NT_data.index.values)
connectors.index = connectors.connector_id

NT_data['skid'] = connectors.loc[np.intersect1d(NT_data.index.values, connectors.index)].presynaptic_to
NT_data['num'] = len(NT_data.index)*[1]
NT_neurons = NT_data.groupby(['skid', 'pred_label'], group_keys=True).count().unstack(fill_value=0).stack()

NT_fractions = NT_neurons/NT_neurons.groupby('skid').transform('sum')
NT_fractions = NT_fractions.unstack('pred_label')
NT_fractions.columns = [x[1] for x in NT_fractions.columns]

# add color based on majority vote
predictions = []
for i in NT_fractions.index:
    if(NT_fractions.loc[i, 'Acetylcholine']>0.5):
        prediction = 'predicted_Ach'
    if(NT_fractions.loc[i, 'GABA']>0.5):
        prediction = 'predicted_GABA'
    if(NT_fractions.loc[i, 'Glutamate']>0.5):
        prediction = 'predicted_Glutamate'
    if((NT_fractions.loc[i, 'Acetylcholine']<=0.5) & (NT_fractions.loc[i, 'GABA']<=0.5) & (NT_fractions.loc[i, 'Glutamate']<=0.5)):
        prediction = 'unclear'
    
    predictions.append(prediction)

NT_fractions['predictions'] = predictions

CN details:   0%|          | 0/170749 [00:00<?, ?it/s]

INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)
INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)
INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)
INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)
INFO  : Data for 170747 of 170749 unique connector IDs retrieved (pymaid)


In [27]:

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

fig = px.scatter_3d(NT_fractions, x='Acetylcholine', y='GABA', z='Glutamate', size=len(NT_fractions.index)*[1], size_max=8, color='predictions', color_discrete_sequence=['gray', 'red', 'purple', 'blue'] , width=1000, height=750)
fig.show()


'\nsns.set(style = "darkgrid")\n\nfig = plt.figure()\nax = fig.add_subplot(111, projection = \'3d\')\n\nx = NT_fractions[\'Acetylcholine\']\ny = NT_fractions[\'GABA\']\nz = NT_fractions[\'Glutamate\']\n\nax.set_xlabel("Acetylcholine")\nax.set_ylabel("GABA")\nax.set_zlabel("Glutamate")\n\nax.scatter(x, y, z)\n\nplt.show()\n'

In [31]:
NT_fractions.groupby('predictions').count().Acetylcholine

predictions
predicted_Ach          1801
predicted_GABA          134
predicted_Glutamate     585
unclear                 487
Name: Acetylcholine, dtype: int64